In [1]:
import tensorflow as tf
import time

In [16]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        time.sleep(0.05)
        for sample_idx in range(num_samples):
            time.sleep(0.03)
            yield (sample_idx,) #Generator
            
    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_files_in_batches,
            output_signature=tf.TensorSpec(shape = (1,),dtype = tf.int64),
            args=(num_samples,)
        )
        pass

In [19]:
def benchmark(dataset, num_epoch=2):
    for epoch_num in range(num_epoch):
        for sample in dataset:
            time.sleep(0.01)

In [20]:
%%timeit
benchmark(FileDataset())

440 ms ± 5.89 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
benchmark(FileDataset().prefetch(1))

387 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

385 ms ± 5.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
dataset = tf.data.Dataset.range(5)
for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [28]:
dataset = dataset.map(lambda x:x**2)
for d in dataset:
    print(d.numpy())

0
1
4
9
16


In [29]:
dataset = dataset.cache()

list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [32]:
def mapped_functions(s):
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [37]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_functions), 5)

1.62 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [38]:
%%timeit -n1 -r1
benchmark(FileDataset().map(mapped_functions).cache(), 5)

518 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
